In [1]:
## load the libraries 
from keras.models import Sequential
from keras.layers import Dense, Input, Conv2D, LSTM, MaxPool2D, UpSampling2D, Flatten, Dropout, concatenate,GlobalAveragePooling2D, AveragePooling2D, ZeroPadding2D, add, Add, LeakyReLU, ReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.utils import to_categorical
from keras.models import Model
from keras.initializers import glorot_uniform, Constant
from keras import optimizers
from keras.layers.core import Activation
from keras.optimizers import Optimizer
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from numpy import argmax, array_equal
from random import randint
import pandas as pd
import numpy as np

C:\Users\Infernal\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv("data/fashion/fashion-mnist_train.csv")
test = pd.read_csv("data/fashion/fashion-mnist_test.csv")

In [3]:
train_x = train[list(train.columns)[1:]].values
train_y = train['label'].values
test_x = test[list(test.columns)[1:]].values
test_y = test['label'].values

In [4]:
train_x=train_x.reshape(60000,28,28,1)
test_x=test_x.reshape(10000,28,28,1)

In [5]:
train_y = to_categorical(train_y, num_classes = 10)
test_y = to_categorical(test_y, num_classes = 10)

In [6]:
train_x = train_x / 255
test_x = test_x / 255

In [7]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1)

In [8]:
datagen = ImageDataGenerator(
        rotation_range=2,
        zoom_range = 0.01,
        width_shift_range=0.03,
        height_shift_range=0.03,
        horizontal_flip=True,
        vertical_flip=False)


datagen.fit(train_x)

In [9]:
def main_block(x, filters, n, strides, activation, initializer, dropout):
    # Normal part
    x_res = Conv2D(filters, (3,3), strides=strides, padding="same", kernel_initializer=initializer, kernel_regularizer=l2(5e-4))(x)
    x_res = BatchNormalization()(x_res)
    x_res = activation()(x_res)
    x_res = Conv2D(filters, (3,3), padding="same", kernel_initializer=initializer)(x_res)
    # Alternative branch
    x = Conv2D(filters, (1,1), strides=strides)(x)
    # Merge Branches
    x = Add()([x_res, x])

    for i in range(n-1):
        # Residual conection
        x_res = BatchNormalization()(x)
        x_res = activation()(x_res)
        x_res = Conv2D(filters, (3,3), padding="same", kernel_initializer=initializer)(x_res)
        # Apply dropout if given
        if dropout: x_res = Dropout(rate=dropout)(x)
        # Second part
        x_res = BatchNormalization()(x_res)
        x_res = activation()(x_res)
        x_res = Conv2D(filters, (3,3), padding="same", kernel_initializer=initializer)(x_res)
        # Merge branches
        x = Add()([x, x_res])

    # Inter block part
    x = BatchNormalization()(x)
    x = activation()(x)
    return x

def build_model(input_dims, output_dim, n, k, activation, initializer, dropout=None):
    assert (n-4)%6 == 0
    assert k%2 == 0
    n = (n-4)//6 
    
    #1)Input
    inputs = Input(shape=(input_dims))

    #2)Initial part
    x = Conv2D(16, (3,3), padding="same", kernel_initializer=initializer)(inputs)
    x = BatchNormalization()(x)
    x = activation()(x)

    #3)wide blocks (can be made into for loop)
    x = main_block(x, 16*k, n, (1,1), activation, initializer, dropout)
    x = main_block(x, 32*k, n, (2,2), activation, initializer, dropout)
    x = main_block(x, 64*k, n, (2,2), activation, initializer, dropout)

    #4)Final part
    x = AveragePooling2D((7,7))(x)
    x = Flatten()(x)
    outputs = Dense(output_dim, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [10]:
model = build_model((28,28,1), 10, 40, 4, ReLU, 'he_uniform',  0.1498182282337851)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 16)   160         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 28, 28, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 28, 28, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
callbacks = [ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5,min_lr=0.00001),
             ModelCheckpoint("best-augmented-model.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')]

model.compile(optimizer=optimizers.Adam(0.00011056751621948101), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit_generator(generator=datagen.flow(train_x, train_y, batch_size=32),
                    validation_data=datagen.flow(val_x, val_y, batch_size=32), 
                    validation_steps=val_x.shape[0] * 2 // 32, steps_per_epoch=train_x.shape[0] * 3 // 32, 
                    epochs = 100, callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
5062/5062 [==============================] - 428s 85ms/step - loss: 0.7274 - acc: 0.8557 - val_loss: 0.5702 - val_acc: 0.8888

Epoch 00001: val_acc improved from -inf to 0.88877, saving model to best-augmented-model.hdf5
Epoch 2/100
5062/5062 [==============================] - 423s 83ms/step - loss: 0.4329 - acc: 0.9112 - val_loss: 0.4019 - val_acc: 0.9116

Epoch 00002: val_acc improved from 0.88877 to 0.91160, saving model to best-augmented-model.hdf5
Epoch 3/100
5062/5062 [==============================] - 423s 84ms/step - loss: 0.3303 - acc: 0.9241 - val_loss: 0.3669 - val_acc: 0.9098

Epoch 00003: val_acc did not improve from 0.91160
Epoch 4/100
5062/5062 [==============================] - 423s 83ms/step - loss: 0.2730 - acc: 0.9324 - val_loss: 0.2960 - val_acc: 0.9245

Epoch 00004: val_acc improved from 0.91160 to 0.92447, saving model to best-augmented-model.hdf5
Epoch 5/100
5062/5062 [==============================] - 4

5062/5062 [==============================] - 424s 84ms/step - loss: 0.0141 - acc: 0.9988 - val_loss: 0.3620 - val_acc: 0.9466

Epoch 00080: val_acc did not improve from 0.95045
Epoch 81/100
5062/5062 [==============================] - 424s 84ms/step - loss: 0.0140 - acc: 0.9988 - val_loss: 0.3515 - val_acc: 0.9470

Epoch 00081: val_acc did not improve from 0.95045
Epoch 82/100
5062/5062 [==============================] - 424s 84ms/step - loss: 0.0140 - acc: 0.9987 - val_loss: 0.3504 - val_acc: 0.9485

Epoch 00082: val_acc did not improve from 0.95045
Epoch 83/100
5062/5062 [==============================] - 424s 84ms/step - loss: 0.0138 - acc: 0.9990 - val_loss: 0.3383 - val_acc: 0.9477

Epoch 00083: val_acc did not improve from 0.95045
Epoch 84/100
5062/5062 [==============================] - 424s 84ms/step - loss: 0.0138 - acc: 0.9989 - val_loss: 0.3513 - val_acc: 0.9471

Epoch 00084: val_acc did not improve from 0.95045
Epoch 85/100
5062/5062 [==============================] - 424s 

In [14]:
y_pred = model.predict(test_x)
accuracy_score(np.argmax(test_y, axis=1), np.argmax(y_pred, axis=1))

0.9527

In [15]:
best_model = build_model((28,28,1), 10, 40, 4, ReLU, 'he_uniform',  0.1498182282337851)

In [16]:
best_model.load_weights("best-augmented-model.hdf5")

In [17]:
y_pred = best_model.predict(test_x)
accuracy_score(np.argmax(test_y, axis=1), np.argmax(y_pred, axis=1))

0.9524